# Tarea 2: Procesamiento de Lenguaje Natural



Integrantes:

* Sebastián Alday (Rut: 18294095-K)
* Paula Canales G. (Rut: 18845155-1)
* Álvaro Neira R. (Rut: 13757209-5)
* Matias Rodriguez U. (Rut: 18362815-1)

# Introducción

In [1]:
!pip install transformers
from transformers import pipeline, AutoTokenizer, Trainer, TrainingArguments, AutoModel, AutoModelForSequenceClassification

     |████████████████████████████████| 3.4 MB 10.0 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 895 kB 60.3 MB/s 
     |████████████████████████████████| 3.3 MB 50.8 MB/s 
     |████████████████████████████████| 61 kB 502 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 1. Creacion y limpieza de la base de datos 

## 1.1 Elija una organización que cuenta con una lista de preguntas y respuestas

Chilexpress
https://centrodeayuda.chilexpress.cl/home

## 1.2 Genere una tabla (tablaQA.xls) con las preguntas y respuestas de la organización seleccionada.

In [2]:
!wget https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv

--2021-12-29 17:38:43--  https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9zty4fuh8boofda/tablaQA.csv [following]
--2021-12-29 17:38:43--  https://www.dropbox.com/s/raw/9zty4fuh8boofda/tablaQA.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc386f7b2e523f67114dcf450b94.dl.dropboxusercontent.com/cd/0/inline/BczWPXsrAOZtLpyG62ZRmGqMq1y9aiPTWKW_oTuSsdYRJYwWq_y8Eo0npRbzjSBfxMBjw7bVeaVw3482onuXmEP3FGUJMrcp5h85B0Jx_hWUhr9VctGbduIxX38XsBGVgs6ugL7LRck4cwyw0NrtYnK0/file# [following]
--2021-12-29 17:38:43--  https://uc386f7b2e523f67114dcf450b94.dl.dropboxusercontent.com/cd/0/inline/BczWPXsrAOZtLpyG62ZRmGqMq1y9aiPTWKW_oTuSsdYRJYwWq_y8Eo0npRbzjSBfxMBjw7bVeaVw3482onuXmEP3FGUJM

In [3]:
import pandas as pd
qa = pd.read_csv('/content/tablaQA.csv', sep=';')
qa.head()

,Pregunta,Respuesta
0,¿Qué es la Orden de Transporte (OT)?,La Orden de Transporte (OT) es el número asoci...
1,¿Cuál es el plazo para retirar un envío desde ...,El plazo para retirar un envío desde una sucur...
2,¿Puede otra persona retirar mi envío de una su...,Para que una persona que no es el destinatario...
3,¿Se puede modificar la dirección de envío?,"Sí, el cambio puede ser solicitado por el remi..."
4,¿Puedo cambiar el destino del envío desde una ...,"Sí, el cambio puede ser solicitado por el remi..."


In [4]:
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pregunta   46 non-null     object
 1   Respuesta  46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


## 1.3 Genere una tabla (tiposmensajes.xls) con ejemplos de las siguientes clases de mensaje: "saludo", "despedida", "nombre", "informacion"

In [5]:
!wget https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv

--2021-12-29 17:39:20--  https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv [following]
--2021-12-29 17:39:20--  https://www.dropbox.com/s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc137a6bed0f55c870160747cc89.dl.dropboxusercontent.com/cd/0/inline/BczviOuOGXQ5ZWfsrgOaxu0o3hzbIJleA0_NdojzBr4sSlXxE4lBWE3HBuQubQ5TRwFGT7bDMIbg5ruqpvw5RzjUqsDkuXWNeU0TIKKeJ8XI3DsmD1f69PQ44563hw6DKfW560UNEDh9hld6RbVzHhMT/file# [following]
--2021-12-29 17:39:21--  https://uc137a6bed0f55c870160747cc89.dl.dropboxusercontent.com/cd/0/inline/BczviOuOGXQ5ZWfsrgOaxu0o3hzbIJleA0_NdojzBr4sSlXxE4lBWE3HBuQubQ5TRwFGT7bDMIb

In [6]:
messages = pd.read_csv('/content/tiposmensajes.csv', sep=',')
messages.head()

,Clase,Mensaje
0,saludo,Hola
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas


## 1.4 Genere respuestas predeterminadas (respuestasDefecto.xls) para los mensajes de tipo "saludo", "despedida", "nombre"

In [7]:
!wget https://www.dropbox.com/s/kukxrqpjxi0l4ei/respuestasDefecto.csv

--2021-12-29 17:40:53--  https://www.dropbox.com/s/kukxrqpjxi0l4ei/respuestasDefecto.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kukxrqpjxi0l4ei/respuestasDefecto.csv [following]
--2021-12-29 17:40:53--  https://www.dropbox.com/s/raw/kukxrqpjxi0l4ei/respuestasDefecto.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc75848f7797303a71a51741fb85.dl.dropboxusercontent.com/cd/0/inline/Bcxl1dlfmO2MwJIgo8ZzZrQywAebO6lr5P1l2VjU5GlBUtBKQ_DZsf6ZPbI0SNhSZvlx_rOAIxyY8hQJKKfup3fh0AwrvLurcdaGe1jfs_6hdg1gvmytEanXyaynZUiD6XZPTIOceVv1jaqOjsh9_VvT/file# [following]
--2021-12-29 17:40:54--  https://uc75848f7797303a71a51741fb85.dl.dropboxusercontent.com/cd/0/inline/Bcxl1dlfmO2MwJIgo8ZzZrQywAebO6lr5P1l2VjU5GlBUtBKQ_DZsf6ZPbI0SN

In [9]:
answers = pd.read_csv('/content/respuestasDefecto.csv', sep=',')
answers.head()

,Clase,Mensaje
1,saludo,"Hola, cuentame qué necetias"
2,saludo,"Buenos días, en qué puedo ayudarte?"
3,saludo,"Buenas tardes, en qué puedo ayudarte?"
4,saludo,"Un gusto saludar, en qué puedo ayudarte?"
5,despedida,Gracias por contactarte con nosotros
6,despedida,"Adios, que esté muy bien"
7,despedida,"Hasta luego, gracias por tu tiempo"
8,despedida,"Adios, espero hayamos resuelto tu inquietud"
9,despedida,"Gracias por tu tiempo, ¡hasta luego!"
10,nombre,"Mi nombre es Eva, estoy aquí para ayudarte"


## 1.5 Describa en términos generales las tablas que construyó.



*   Tabla QA: Se construyó en base a las preguntas y respuestas del centro de ayuda de Chilexpress https://centrodeayuda.chilexpress.cl/home . Se extrajo cada pregunta y respuesta de cada categoría (Envíos Nacionales, Envíos y retiros en App, Envíos internacionales, Giros en Chile y el mundo, Atención en sucursal, Cuenta empresa) y se eliminaron las preguntas repetidas. 
*   Tipos de mensajes: se construyó con mensajes tipo de saludo, despedida y nombre. Para la información se utilizó las preguntas obtenidas en la Tabla QA.
*   Respuestas por defecto: se construyó con respuestas tipo de saludo, despedida y nombre de nuestro bot Eva



# 2. Análisis de distancia

Para el primer chatbot utilizará una métrica de su elección para responder
a cada pregunta/texto del usuario.

## 2.1 Utilice algún embeddings utilizado en el curso para codificar el texto de entrada

### 2.1.1 Word2Vec

In [10]:
import nltk
import gensim
import re
import pandas as pd
import sklearn
import numpy as np

import sklearn.linear_model
import sklearn.model_selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [11]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  #if len(tokens)==1:
  #  tokens=text
  return tokens

def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

In [12]:
corpus = messages.Mensaje.tolist()
corpus_preprocessed = list(map(preprocessor,corpus))


In [13]:
word2vec_model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed, min_count=1)

## 2.2 Con el texto de entrada codificado y usando embeddings proponga una manera de identificar la clase del texto de entrada.

In [14]:
def predict_class(question):
  tokens = preprocessor(question)
  vector = vectorizer(tokens, word2vec_model).reshape(1, -1)
  return clf.predict(vector)

### 2.2.1 Word2Vec

In [15]:
features = np.zeros(shape=(len(messages), word2vec_model.wv.vectors.shape[1]))
for i,msg in enumerate(messages.Mensaje):
  features[i,:] = vectorizer(preprocessor(msg), word2vec_model)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.7666666666666667

In [16]:
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(features,np.array(messages.Clase))

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [17]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Mi nombre es'))
print(predict_class('hola buenas'))
print(predict_class('hola'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))
print(predict_class('hasta luego'))
print(predict_class('adios'))

['información']
['nombre']
['saludo']
['saludo']
['información']
['información']
['despedida']
['despedida']


## 2.3 Si el texto de entrada es del tipo "información”, busque ahora la pregunta mas similar y retorne la respuesta asociada.

In [18]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
pattern = r"""(?x)                   # set flag to allow verbose regexps
              (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A.
              |\$?\d+(?:[.,]\d+)?%?  # numbers, incl. currency and percentages
              |\w+(?:[-']\w+)*       # words w/ optional internal hyphens/apostrophe
              |(?:[+/\-@&*¡!.,])     # special characters with meanings
            """

def normalizer(text, remove_tildes = True): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = nltk.regexp_tokenize(text, pattern)
    text = ' '.join(text)
    if remove_tildes:
        text = re.sub('á', 'a', text) #replaces special vowels to their base forms
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [20]:
nltk_stopwords = nltk.corpus.stopwords.words('spanish')
stopwords_normalized = [normalizer(word) for word in nltk_stopwords]

def no_stopwords(processed):
  '''
  Función para eliminar las stopwords de un texto
  '''
  result = []
  for w in processed:
    if w not in stopwords_normalized:
      result.append(w)

  return result

In [21]:
corpus_question = qa.Pregunta.tolist()
questions_processed = []
for question in corpus_question:
  pre = preprocessor(question)
  pre = no_stopwords(pre)
  questions_processed.append(' '.join(pre))

In [22]:
questions_processed[:5]

['orden transporte ot',
 'plazo retirar envio sucursal chilexpress',
 'puede persona retirar envio sucursal chilexpress',
 'puede modificar direccion envio',
 'puedo cambiar destino envio direccion particular sucursal chilexpress']

ref: https://www.linkedin.com/pulse/how-find-semantic-similar-sentences-from-your-dataset-jayaraman/

In [23]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(questions_processed)]

# hyper parameters
max_epochs = 2000
vec_size = 200
alpha = 0.1
minimum_alpha = 0.025
reduce_alpha = 0.002

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=minimum_alpha,
                dm =1)
model.build_vocab(tagged_data)

# Train the model based on epochs parameter
for epoch in range(max_epochs):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)

# Save model. 
model.save("similar_sentence.model3")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [24]:
model= Doc2Vec.load("similar_sentence.model3")

def best_five_answers(question):
  tokens = preprocessor(question)
  tokens = no_stopwords(tokens)
  vector = model.infer_vector(tokens)
  most_similar = model.docvecs.most_similar([vector]) 
  answers = qa.Respuesta.tolist()
  print(tokens)
  for index,value in most_similar[:5]:
    print('---------')
    print(corpus_question[int(index)])
    print(questions_processed[int(index)])
    print(answers[int(index)])
    print('---------')

def best_answers(question):
  tokens = preprocessor(question)
  tokens = no_stopwords(tokens)
  vector = model.infer_vector(tokens)
  most_similar = model.docvecs.most_similar([vector]) 
  answers = qa.Respuesta.tolist()
  for index,value in most_similar[:1]:
    print(corpus_question[int(index)])
    print(answers[int(index)])

In [25]:
best_five_answers('¿Qué es la Orden de Transporte (OT)?')

['orden', 'transporte', 'ot']
---------
Hice un giro online pero ingresé mal el nombre del destinatario ¿cómo puedo corregirlo?
hice giro online ingrese mal nombre destinatario puedo corregirlo
Para corregir el nombre del destinatario de un giro online ,debes ingresar a https://te-ayudamos.chilexpress.cl/. Luego seleccionar "ingresa un reclamo", posteriormente seleccionar "envíos de dinero" y luego "quiero modificar los datos de mi envío de dinero".

---------
---------
Si hice un Giro online y no he recibido el MTCN, ¿puedo solicitarlo por teléfono?
hice giro online recibido mtcn puedo solicitarlo telefono
Si no recibiste tu MTCN vía SMS o por correo electrónico ahora puedes conocer el estado, historial y número de MTCN de todos tus envíos de dinero en la sección Mis Giros en www.chilexpress.cl

---------
---------
¿Qué pasa si el destinatario no puede recibir el envío en su dirección?
pasa destinatario puede recibir envio direccion
Chilexpress realiza dos intentos de entrega en el do

In [26]:
best_five_answers('¿Qué es un envío sobredimensionado?')

['envio', 'sobredimensionado']
---------
Si el destinatario no se encuentra en la dirección de envío, ¿puede recibirlo otra persona?
destinatario encuentra direccion envio puede recibirlo persona
Sí, puede recibirlo cualquier persona adulta que esté en el domicilio, indicando su Rut y su nombre.

---------
---------
¿El destinatario puede cambiar la dirección del envío?
destinatario puede cambiar direccion envio
No es posible. Sin embargo, si un envío no puede ser entregado en la dirección de destino original, el destinatario podrá solicitar que sea derivado a una sucursal Chilexpress ubicada en la misma comuna del despacho original.

---------
---------
¿Qué pasa si el destinatario no puede recibir el envío en su dirección?
pasa destinatario puede recibir envio direccion
Chilexpress realiza dos intentos de entrega en el domicilio, en dos días hábiles y horarios distintos, los que son notificados al destinatario mediante un email. Si no se puede efectuar la entrega, el envío será devue

## 2.4 Reporte el resultado con textos de prueba.

# 3. Análisis transformer 

Para el segundo chatbot utilice un transformer (por ejemplo BERT).

## 3.1 Utilice el modelo transformer para clasificar el texto de entrada, y para extraer la respuesta de la tabla de preguntas y respuestas cuando el mensaje sea del tipo “información”.

### 3.1.1 Clasificador

<font color='red'>**Pendiente**</font>.

### 3.1.2 Extraccion de respuestas para el caso "informacion"

In [27]:
# Transformer QA
beto_qa = pipeline('question-answering', 
        model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
        tokenizer=('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es', {"use_fast": False})
    )


Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/237k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [28]:
context_text = ""
for index, row in qa.iterrows():
  context_text = context_text + ". " + row['Respuesta']

In [29]:
def beto_get_answer(question):
  respuesta = beto_qa({'question': question, 'context': context_text})
  if respuesta['score'] < 0.01:
    return "Perdón, pero no tengo respuesta para esa pregunta."
  else:
    return respuesta['answer']

## 3.2 Reporte el tipo de red, y las métricas de entrenamiento usadas

## 3.3 Reporte el resultado con los textos de prueba.

In [30]:
print(beto_get_answer('¿Sólo las empresas pueden tener una cuenta con Chilexpress?')+"\n")
print(beto_get_answer('¿puedo retirar un giro de dinero?')+"\n")
print(beto_get_answer('Puede otra persona retirar mi pedido')+"\n")
print(beto_get_answer('¿Qué es la Orden de Transporte (OT)?')+"\n")
print(beto_get_answer('¿Qué es un envío sobredimensionado?')+"\n")
print(beto_get_answer('¿Quién ganará la próxima elección?')+"\n")

Sí,

Perdón, pero no tengo respuesta para esa pregunta.

Una tercera persona puede retirar un envío por pagar

el número asociado al envío.

embalaje irregular,

Perdón, pero no tengo respuesta para esa pregunta.



# Conclusión